In [1]:
import pandas as pd
import cufflinks
from tqdm.auto import tqdm
tqdm.pandas()

cufflinks.go_offline()

/home/memesrized/.virtualenvs/edu/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



In [7]:
import syntok.segmenter as segmenter 
import re

def repetitions(s):
    r = re.compile(r"(.+?)\1+")
    for match in r.finditer(s):
        yield (match.group(1), len(match.group(1)), len(match.group(0))/len(match.group(1)))

def filter_jigsaw(toxic):
    toxic = toxic[toxic.map(lambda x: "[[" not in x and "{{" not in x and not "]]" in x and not "}}" in x)]
    toxic = toxic.map(lambda x: x[1:-1] if x.startswith('"') else x)
    toxic = toxic.map(lambda x: re.sub(r"\:\:+", " ", x))
    toxic = toxic.map(lambda x: x.replace('""', '"').replace('""', '"').replace("'''", " "))
    toxic = toxic[~toxic.map(lambda x: [z for z in repetitions(x) if z[1]>3 and z[-1]>1]).map(bool)]
    toxic = toxic.map(lambda x: re.sub(r"italic text|bold text", " ", x, flags=re.IGNORECASE).strip())
    toxic = toxic.map(lambda x: re.sub(r"\d+\.\d+\.\d+\.\d+", " ", x).strip())
    toxic = toxic.map(lambda x: re.sub(r"\bTAB\b|TABTAB", " ", x))
    toxic = toxic.map(lambda x: re.sub(r"!+", "!", x))
    toxic = toxic.map(lambda x: re.sub(r"\?+", "?", x))
    toxic = toxic.map(lambda x: re.sub(r"\/+", r"/", x))

    toxic = toxic.map(lambda x: re.sub(r"\s+", ' ', x).strip())
    toxic = toxic[
        toxic.map(
            lambda x: bool(re.match(r"[A-Z]", x[0])) and bool(re.match(r"[.?!]", x[-1]) and not x.isupper())
        )
    ]
    toxic = toxic[toxic.str.len()<250]
    toxic = toxic[toxic.map(lambda x: True if len(list(list(segmenter.process(x))[0]))<9 else False)]
    toxic = toxic.drop_duplicates()
    return toxic

In [2]:
df = pd.read_csv("data/jigsaw/jigsaw.csv")

df_t = pd.read_csv("data/jigsaw/test.csv")

len(df), len(df_t)

(159571, 153164)

In [3]:
df['toxic'].sum()

15294

In [8]:
df = pd.read_csv("data/jigsaw/jigsaw.csv")

df_t = pd.read_csv("data/jigsaw/test.csv")
df_t_labels = pd.read_csv("data/jigsaw/test_labels.csv")
df_t = pd.merge(df_t, df_t_labels, on="id")
toxic_t = df_t[df_t['toxic']==1]['comment_text']
len(toxic_t)

6090

In [9]:
toxic_t = filter_jigsaw(toxic_t)
toxic = filter_jigsaw(df[df['toxic'].map(bool)]['comment_text'])

In [10]:
len(toxic), len(toxic_t)

(5086, 1193)

In [11]:
toxic_full = toxic.append(toxic_t)

In [12]:
len(toxic_full)

6279

In [213]:
toxic_full.to_csv("data/jigsaw/jigsaw_toxic_full.csv")

In [231]:
for x in (segmenter.("Hi, i'm mark. What are you gonna do?")):
    print(x)

TypeError: 'str' object is not an iterator

In [224]:
list(list(segmenter.segment("Hi, i'm mark. What are you gonna do?"))[0])

TypeError: 'str' object is not an iterator